In [95]:
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path
from typing import List

import utm
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry as sg
import shapely.ops as so

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def embed_map(m):
    """
    Workaround taken from https://github.com/python-visualization/folium/issues/812
    for displaying Folium maps with lots of features in Chrome-based browsers.
    """
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

In [31]:
path = DATA_DIR/'cairns_gtfs.zip'
feed = (
    gk.read_gtfs(path, dist_units='km')
    .append_dist_to_stop_times()
)

/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [174]:
def bingo(
    feed: "Feed", screen_lines: gpd.GeoDataFrame, dates: List[str]
) -> pd.DataFrame:
    """
    Find all the Feed trips active on the given YYYYMMDD dates whose shapes
    intersect the given GeoDataFrame of screen lines, that is, of straight WGS84 LineStrings.
    Compute the intersection times and directions for each trip.

    Return a DataFrame with the columns

    - ``'date'``
    - ``'trip_id'``
    - ``'route_id'``
    - ``'route_short_name'``
    - ``'crossing_time'``: time that the trip's vehicle crosses
      the linestring; one trip could cross multiple times
    - ``'crossing_direction'``: 1 or -1; 1 indicates trip travel from the
      left side to the right side of the screen line;
      -1 indicates trip travel in the  opposite direction

    Notes:
    
    - Assume the Feed's stop times DataFrame has an accurate ``shape_dist_traveled`` column.
    - Assume that trips travel in the same direction as their shapes, an assumption
      that is part of the GTFS.
    - Assume that the screen line is straight and simple.
    - Probably does not give correct results for trips with self-intersecting shapes.
    - The algorithm works as follows

        1. Find the trip shapes that intersect the screen lines.
        2. For each such shape and screen line, compute the intersection points, the distance
           of the point along the shape, and the orientation of the screen line relative to
           the shape.
        3. For each given date, restrict to trips active on the
           date and interpolate a stop time for the intersection point using
           the ``shape_dist_traveled`` column.
        4. Use that interpolated time as the crossing time of the trip vehicle.

    """
    dates = feed.subset_dates(dates)
    if not dates:
        return pd.DataFrame()

    # Get shapes as GeoDataFrame
    shapes_g = feed.geometrize_shapes(use_utm=True)

    # Convert screen lines to UTM
    crs = shapes_g.crs
    screen_lines = screen_lines.to_crs(crs)
    
    # Create screen line IDs if necessary
    n = screen_lines.shape[0]
    if "screen_line_id" not in screen_lines.columns:
        screen_lines["screen_line_id"] = gk.make_ids(n, "sl")
    
    # Make a vector in the direction of each screen line to calculate crossing orientation.
    # Does not work in case of a bent screen line.
    p1 = screen_lines.geometry.map(lambda x: np.array(x.coords[0]))
    p2 = screen_lines.geometry.map(lambda x: np.array(x.coords[-1]))
    screen_lines["screen_line_vector"] = p2 - p1
    
    # Get intersection points of shapes and screen lines
    g0 = (
        # Only keep shapes that intersect screen lines to reduce computations
        gpd.sjoin(shapes_g, screen_lines.filter(["screen_line_id", "geometry"]))
        .merge(screen_lines, on="screen_line_id")
        .assign(geometry_x=lambda x: gpd.GeoSeries(x.geometry_x, crs=crs))
        .set_geometry("geometry_x")
        # Compute intersection points
        .assign(int_point=lambda x: x.geometry_x.intersection(
            gpd.GeoSeries(x.geometry_y, crs=crs)))
    )
    
    # Unpack multipoint intersections to yield a new GeoDataFrame
    records = []
    for row in g0.itertuples(index=False):
        if isinstance(row.int_point, sg.Point):
            intersections = [row.int_point]
        else:
            intersections = row.int_point
        for int_point in intersections:
            record = {
                "shape_id": row.shape_id,
                "screen_line_id": row.screen_line_id,
                "geometry": row.geometry_x,
                "int_point": int_point,
                "screen_line_vector": row.screen_line_vector,
            }
            records.append(record)

    g = gpd.GeoDataFrame.from_records(records)
    g.crs = crs

    # Get distance (in meters) of each intersection point along shape
    g["crossing_dist"] = g.apply(lambda x: x.geometry.project(x.int_point), axis=1)
    
    # Build a tiny vector along each shape
    p2 = g.apply(lambda x: x.geometry.interpolate(x.crossing_dist + 1), axis=1).map(lambda x: np.array(x.coords[0]))
    p1 = g.int_point.map(lambda x: np.array(x.coords[0]))
    g["shape_vector"] = p2 - p1
    
    # Compute crossing direction by taking the vector cross product of 
    # the shape vector and the screen line vector
    det = g.apply(lambda x: np.linalg.det(np.array([x.shape_vector, x.screen_line_vector])), axis=1)
    g["crossing_direction"] = det.map(lambda x: 1 if x >=0 else -1)

    # Convert to feed distance units
    converter = gk.get_convert_dist("m", feed.dist_units)
    g["crossing_dist"] = g["crossing_dist"].map(converter)

    # Summarize work so far into a lookup table
    h = (
        g
        .filter(["shape_id", "screen_line_id", "crossing_direction", "crossing_dist"])
        .set_index("shape_id")
        .sort_values(["shape_id", "crossing_dist"])  # Need this sorting for interpolation to work
    )
    
    # Get stop times of trips whose shapes lie in h
    st = (
        feed.trips.loc[lambda x: x.shape_id.isin(h.index)]
        # Merge in route short names and stop times
        .merge(feed.routes[["route_id", "route_short_name"]])
        .merge(feed.stop_times)
        # Keep only non-NaN departure times 
        .loc[lambda x: x.departure_time.notna()]
        # Convert to seconds past midnight
        .assign(departure_time=lambda x: x.departure_time.map(gk.timestr_to_seconds))
    )

    # Compute crossing times by date
    records = []
    ta = feed.compute_trip_activity(dates)
    for date in dates:        
        # Subset to trips active on date and merge with g
        ids = ta.loc[lambda x: x[date] == 1, "trip_id"]
        f = (
            st.loc[lambda x: x.trip_id.isin(ids)]
            .sort_values(["trip_id", "shape_dist_traveled"])  # Need this sorting for interpolation to work
        )
        
        # Get crossing time for each trip
        for tid, group in f.groupby("trip_id"):
            sid = group["shape_id"].iat[0]
            rid = group["route_id"].iat[0]
            rsn = group["route_short_name"].iat[0]
            dists = group["shape_dist_traveled"].values
            times = group["departure_time"].values
            crossing_dists = h.loc[[sid], "crossing_dist"].values
            crossing_times = np.interp(crossing_dists, dists, times)
            for i, row in enumerate(h.loc[[sid]].itertuples(index=False)):
                record = {
                    "date": date,
                    "trip_id": tid,
                    "route_id": group.route_id.iat[0],
                    "route_short_name": group.route_short_name.iat[0],
                    "shape_id": group.shape_id.iat[0],
                    "screen_line_id": row.screen_line_id,
                    "crossing_direction": row.crossing_direction,
                    "crossing_distance": row.crossing_dist,
                    "crossing_time": crossing_times[i],
                }
                records.append(record)
                
    result = (
        pd.DataFrame.from_records(records)
        .assign(crossing_time=lambda x: x.crossing_time.map(
            lambda x: gk.timestr_to_seconds(x, inverse=True)
        ))
    )
    return result


In [175]:
collection = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            145.7731032371521,
            -16.917382184901566
          ],
          [
            145.7735753059387,
            -16.917053719034044
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            145.76706290245056,
            -16.916376256373532
          ],
          [
            145.76752424240112,
            -16.915986201009297
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            145.75278282165527,
            -16.912598844152377
          ],
          [
            145.7535982131958,
            -16.914076970997165
          ]
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "LineString",
        "coordinates": [
          [
            145.7585334777832,
            -16.903072859722965
          ],
          [
            145.75737476348877,
            -16.90405832871205
          ]
        ]
      }
    }
  ]
}
screen_lines = gpd.GeoDataFrame.from_features(collection["features"], crs=gk.WGS84)
display(screen_lines)

dates = feed.get_first_week()[:1]
%time f1 = bingo(feed, screen_lines.iloc[[0]], dates)
print(type(f1))
display(f1)

geometry
0  LINESTRING (145.77310 -16.91738, 145.77358 -16...
1  LINESTRING (145.76706 -16.91638, 145.76752 -16...
2  LINESTRING (145.75278 -16.91260, 145.75360 -16...
3  LINESTRING (145.75853 -16.90307, 145.75737 -16...

/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


CPU times: user 986 ms, sys: 0 ns, total: 986 ms
Wall time: 984 ms
<class 'pandas.core.frame.DataFrame'>


date                             trip_id route_id route_short_name  \
0    20140526  CNS2014-CNS_MUL-Weekday-00-4165878  110-423              110   
1    20140526  CNS2014-CNS_MUL-Weekday-00-4165879  110-423              110   
2    20140526  CNS2014-CNS_MUL-Weekday-00-4165880  110-423              110   
3    20140526  CNS2014-CNS_MUL-Weekday-00-4165881  110-423              110   
4    20140526  CNS2014-CNS_MUL-Weekday-00-4165882  110-423              110   
..        ...                                 ...      ...              ...   
312  20140526  CNS2014-CNS_MUL-Weekday-00-4172816  123-423              123   
313  20140526  CNS2014-CNS_MUL-Weekday-00-4172817  123-423              123   
314  20140526  CNS2014-CNS_MUL-Weekday-00-4172818  123-423              123   
315  20140526  CNS2014-CNS_MUL-Weekday-00-4172819  123-423              123   
316  20140526  CNS2014-CNS_MUL-Weekday-00-4172820  123-423              123   

    shape_id screen_line_id  crossing_direction  crossing_distance  \
0    1100023            sl0                   1          31.279921   
1    1100023            sl0                   1          31.279921   
2    1100023            sl0                   1          31.279921   
3    1100023            sl0                   1          31.279921   
4    1100023            sl0                   1          31.279921   
..       ...            ...                 ...                ...   
312  1230067            sl0                  -1           1.496863   
313  1230067            sl0                  -1           1.496863   
314  1230067            sl0                  -1           1.496863   
315  1230067            sl0                  -1           1.496863   
316  1230067            sl0                  -1           1.496863   

    crossing_time  
0        06:44:29  
1        07:14:29  
2        07:44:29  
3        08:14:29  
4        08:44:29  
..            ...  
312      14:14:09  
313      15:14:09  
314      16:14:09  
315      17:14:09  
316      18:14:09  

[317 rows x 9 columns]

In [176]:
%%time

for __ in range(screen_lines.shape[0]):
    f2 = feed.compute_screen_line_counts(screen_lines.geometry.iat[0], dates)

/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/home/araichev/.virtualenvs/gtfs_kit/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_s

CPU times: user 2.01 s, sys: 15.7 ms, total: 2.02 s
Wall time: 2.02 s


In [177]:
g1 = f1.loc[lambda x: x.screen_line_id == "sl0"].sort_values("trip_id")
g2 = f2.sort_values("trip_id")

In [179]:
(
    g1.crossing_time.equals(g2.crossing_time),
    g1.crossing_direction.equals(g2.orientation)
)

(True, True)

In [ ]:
embed_map(feed.map_routes(feed.routes.route_id.iloc[:4], include_stops=False))
